In [12]:
import tensorflow as tf
from tensorflow.python.ops.rnn_cell import LSTMCell
import numpy as np
tf.set_random_seed(2016)
np.random.seed(2016)

tf.reset_default_graph()
sess=tf.Session()

In [53]:
wordslist = np.load('word_list.npy')
wordslist = wordslist.tolist()
wordslist = [word.decode('UTF-8') for word in wordslist]
wordvectors = np.load('word_embeddings.npy')
print(len(wordslist))
print(wordvectors.shape)

2941
(2941, 300)


In [54]:
#batch_size=1
max_seq_length=40
#iteration=1000
#word2wec_dim=50
batchsize = 10
iterations = 10000
#PAD = 0
EOS = -1
input_embedding_size = 300
encoder_hidden_units = 4096
decoder_hidden_units = encoder_hidden_units
encoder_inputs = tf.placeholder(shape=(batchsize,max_seq_length), dtype=tf.int32, name='encoder_inputs')
decoder_targets = tf.placeholder(shape=(batchsize, max_seq_length), dtype=tf.int32, name='decoder_targets')
decoder_inputs = tf.placeholder(shape=(batchsize, max_seq_length), dtype=tf.int32, name='decoder_inputs')
print(encoder_inputs.shape)
print(decoder_targets.shape)
print(decoder_inputs.shape)

(10, 40)
(10, 40)
(10, 40)


In [55]:
ids = np.load('idsMatrix.npy')
decoder_inp = np.load('decoder-input-matrix.npy')
decoder_out = np.load('decoder-target-matrix.npy')
print(decoder_inp.shape)
print (np.amax(ids))
print(ids)

(40000, 40)
2940
[[1663 1134  357 ...    0    0    0]
 [1134  357 2414 ...    0    0    0]
 [1287  298 2035 ...    0    0    0]
 ...
 [1663  752 1067 ...    0    0    0]
 [1663  357 2134 ...    0    0    0]
 [1663  357 1864 ...    0    0    0]]


In [37]:
ids.dtype

dtype('int32')

In [56]:
wordvectors.dtype

dtype('float32')

In [7]:
wordvectors = wordvectors.astype(dtype=np.float32)

In [57]:
numDimensions = 300
encoder_inputs_embedded = tf.Variable(tf.zeros([batchsize, max_seq_length, numDimensions]),dtype=tf.float32)
decoder_inputs_embedded = tf.Variable(tf.zeros([batchsize, max_seq_length, numDimensions]),dtype=tf.float32)
decoder_outputs_embedded = tf.Variable(tf.zeros([batchsize, max_seq_length, numDimensions]),dtype=tf.float32)
#print(encoder_inputs_embedded.shape)

encoder_inputs_embedded = tf.nn.embedding_lookup(wordvectors, encoder_inputs)
decoder_inputs_embedded = tf.nn.embedding_lookup(wordvectors, decoder_inputs)
decoder_outputs_embedded = tf.nn.embedding_lookup(wordvectors, decoder_targets)
print(encoder_inputs_embedded.shape)
print(decoder_inputs_embedded.shape)
print(decoder_outputs_embedded.shape)

(10, 40, 300)
(10, 40, 300)
(10, 40, 300)


In [58]:
encoder_cell = tf.contrib.rnn.LSTMCell(encoder_hidden_units)
#encoder_cell.shape

In [59]:
encoder_inputs_embedded

<tf.Tensor 'embedding_lookup_3:0' shape=(10, 40, 300) dtype=float32>

In [60]:
encoder_outputs, encoder_final_state = tf.nn.dynamic_rnn(encoder_cell, encoder_inputs_embedded,dtype=tf.float32)
print(encoder_outputs.shape)
#print(encoder_final_state.shape)

(10, 40, 4096)


In [61]:
encoder_cell

In [62]:
encoder_outputs

<tf.Tensor 'rnn/transpose_1:0' shape=(10, 40, 4096) dtype=float32>

In [63]:
encoder_final_state

LSTMStateTuple(c=<tf.Tensor 'rnn/while/Exit_3:0' shape=(10, 4096) dtype=float32>, h=<tf.Tensor 'rnn/while/Exit_4:0' shape=(10, 4096) dtype=float32>)

In [64]:
decoder_cell = tf.contrib.rnn.LSTMCell(decoder_hidden_units)

In [65]:
decoder_outputs, decoder_final_state = tf.nn.dynamic_rnn(
    decoder_cell, decoder_inputs_embedded,
    initial_state=encoder_final_state,
    dtype=tf.float32, time_major=False, scope="plain_decoder")

In [66]:
decoder_outputs

<tf.Tensor 'plain_decoder/transpose_1:0' shape=(10, 40, 4096) dtype=float32>

In [67]:
decoder_final_state

LSTMStateTuple(c=<tf.Tensor 'plain_decoder/while/Exit_3:0' shape=(10, 4096) dtype=float32>, h=<tf.Tensor 'plain_decoder/while/Exit_4:0' shape=(10, 4096) dtype=float32>)

In [68]:
decoder_logits = tf.contrib.layers.linear(decoder_outputs, len(wordslist))
decoder_prediction = tf.argmax(decoder_logits, 2)

In [69]:
decoder_logits

<tf.Tensor 'fully_connected/BiasAdd:0' shape=(10, 40, 2941) dtype=float32>

In [70]:
decoder_prediction

<tf.Tensor 'ArgMax:0' shape=(10, 40) dtype=int64>

In [71]:
stepwise_cross_entropy = tf.nn.softmax_cross_entropy_with_logits(
    labels=tf.one_hot(decoder_targets, depth= len(wordslist), dtype=tf.float32),
    logits=decoder_logits,
)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [72]:
loss = tf.reduce_mean(stepwise_cross_entropy)
train_op = tf.train.AdamOptimizer().minimize(loss)

In [73]:
stepwise_cross_entropy

<tf.Tensor 'softmax_cross_entropy_with_logits_sg/Reshape_2:0' shape=(10, 40) dtype=float32>

In [25]:
ids = np.load('idsMatrix.npy')

decoder_inp = np.load('decoder-input-matrix.npy')
decoder_out = np.load('decoder-target-matrix.npy')

In [26]:
import random
#def get_next(matrix):
def next_feed(current_index):
    k = 0
    encoderinputs = list()
    decoderinputs = list()
    decodertargets = list()
    for i in range(current_index,current_index+100):
        val = random.randint(current_index,current_index+5)
        encoderinputs.append(ids[val])
        decoderinputs.append(decoder_inp[val])
        decodertargets.append(decoder_out[val])
        i = i + 5
    current_index = current_index + 500
    return{
        encoder_inputs: np.asarray(encoderinputs,dtype=np.int),
        decoder_inputs: np.asarray(decoderinputs,dtype=np.int),
        decoder_targets: np.asarray(decodertargets,dtype=np.int)
    }

In [75]:
import random
def get_batch():
    import random
    value = random.randint(0,39991)
    
    matrix = ids[value:(value+10)]
    matrix1 = decoder_inp[value:(value+10)]
    matrix2 = decoder_out[value:(value+10)]
    return{
        encoder_inputs: matrix,
        decoder_inputs: matrix1,
        decoder_targets: matrix2
    }


In [81]:


sess.run(tf.global_variables_initializer())
saver = tf.train.Saver()
loss_track = []


current_index = 0



n_steps = 4001
batches_in_epoch = 40

try:
    for batch in range(1):
        fd = get_batch()
        #print(fd)
        current_index = current_index + 500
        _, l = sess.run([train_op, loss],fd)
        loss_track.append(l)

        if batch == 0 or batch % batches_in_epoch == 0:
            print('batch {}'.format(batch))
            print('  minibatch loss: {}'.format(sess.run(loss, fd)))
            #print('  minibatch loss: {}'.format(sess.run(loss, fd)))
            predict_ = sess.run(decoder_prediction, fd)
            for i, (inp, pred) in enumerate(zip(fd[encoder_inputs].T, predict_.T)):
                print('  sample {}:'.format(i + 1))
                print('    input     > {}'.format(inp))
                print('    predicted > {}'.format(pred))
                if i >= 2:
                    break
        
    
    print(save_path)
except KeyboardInterrupt:
    print('training interrupted')
    
save_path = saver.save(sess, "/media/Hardik/58744D2F744D10E4/tmp/model.ckpt")
#sess.close()

batch 0
  minibatch loss: 4.43425655365
  sample 1:
    input     > [1663 1663 1663 1663 1663 1663 2002 1663 1663 1663]
    predicted > [1663 1663 1663 1663 1663 1663 1663 1663 1663 1663]
  sample 2:
    input     > [1516 1516 2002 2002 1533 2239  376 2002 2002  468]
    predicted > [0 0 0 0 0 0 0 0 0 0]
  sample 3:
    input     > [1675 2283  488 1428  468  468 2488  492  376  376]
    predicted > [0 0 0 0 0 0 0 0 0 0]
tmp/model.ckpt
